# Similarity metrics for registration

Here, we will learn about similarity metrics.

Specifically, we will compare two popular metrics: 

* [Mean squared error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error#Predictor)
* [Normalized mutual information (NMI)](https://en.wikipedia.org/wiki/Mutual_information#Normalized_variants)

In this example, we'll first observes that MSE fails, and then understand why it fails. Then observe that using NMI succeeds, and understand why that is the case.

First, let's import libraries and make some useful functions:

In [ ]:
import numpy as np
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

%matplotlib inline
import matplotlib.pyplot as plt

def compare_overlay(moving, fixed):
    farr = sitk.GetArrayFromImage(fixed).astype(np.float32)
    rarr = sitk.GetArrayFromImage(moving).astype(np.float32)

    plt.figure()
    plt.imshow(farr, 'gray', interpolation='none', alpha=0.8)
    plt.imshow(rarr, 'winter', interpolation='none', alpha=0.5)
    
def run_elastix(fixed_image, moving_image, params):
    """
    Runs elastix with the given fixed and moving images,
    and elastix parameters.
    
    Returns the ElastixImageFilter, from which the results
    can be obtained.
    """
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

Read the example images and visualize them in three different ways:

In [ ]:
movingImage = sitk.ReadImage('../sampleImages/ct_2d_moving.tif')
fixedImage  = sitk.ReadImage('../sampleImages/mri_2d.tif')

First we'll simply visualize the images side-by-side, with `itkwidgets` as we did in the `translation_initialization` notebook.

The CT image (moving) is on the left, and the MR image (fixed) is on the right. Notice:

* Features of the brain are visible in the MR but not the CT
* The skull is bright in the CT but dark in the MR.

In [ ]:
itkwidgets.compare(movingImage,fixedImage)

Next we'll use the "checkerboard" visualization that comes with `itkwidgets`.  Try changing the `pattern` parameter to 12, for example.

In [ ]:
itkwidgets.checkerboard(movingImage,fixedImage, pattern=12)

Last, let's overlay the images using the function defined above.

The main point is that the images are not currently well aligned.

In [ ]:
compare_overlay(movingImage, fixedImage)

Let's load a set of elastix parameters from a file.

In [ ]:
params_mse = sitk.ReadParameterFile('../elastixParameters/AffineMSE_2d.txt')
sitk.PrintParameterMap( params_mse )

Notice the line `(Metric "AdvancedMeanSquares")`, indicating that elastix will use mean squared error. The [documentation](https://elastix.lumc.nl/doxygen/classelastix_1_1AdvancedMeanSquaresMetric.html).

Now, run elastix with these parameters, and visualize the results by overlaying the images.

In [ ]:
elastix_mse = run_elastix(fixedImage, movingImage, params_mse)
result_mse = elastix_mse.GetResultImage()

In [ ]:
compare_overlay(result_mse, fixedImage)

The result is not great. At the bottom of the image, the skull in the CT is overlayed with part of the brain in the MR. But near the top of the image, the skull is too far up.  As well, there is some rotation that was not corrected for.

Let's understand why Mean Squared Error produced this result. MSE wants the images to be exactly identical after the transformation is applied. I.e. it wants low-intensities in one image to correspond to low intensities in the other image. Similarly, it tries to match locations of high intensities in both images.

However, in this example, our goal is for some high intensities (skull) in the CT to correspond to some low intensities in the MR, which is penalized highly by MSE. This explains why MSE failed - because it penalized strongly our desired transformation, and found a different one that matches better its goal. (Notice the bright skull is overlayed with the moderately bright brain
near the bottom of the resulting image).

To make the algorithm work, we need to find a metric that aligns with our goal. Cross correlation is another popular metric,  but it also tries to match high intensities to each other and low intensities to each other. They need not be identical, so it is less strict than MSE.

Mutual information is a third popular metric, and will be what we try next. It is the least strict of the three. Roughly, it measures how much knowing the intensity of one image tells you about the intensity of the other image at the same location.

Consider the result above. The brightest intensities for the CT are the skull. Those locations are aligned with some medium intensities, and some low intensities in the MR. So it would be hard to predict the MR intensity knowing the CT intensity is bright. When the images are aligned, the bright intensities in CT will almost always correspond to low intensities in the MR. I.e. knowing the CT intensity is bright would give us lots of information about the intensity of the MR image. 

Now let's try mutual information, by first loading a parameters file:

In [ ]:
params_mi = sitk.ReadParameterFile('../elastixParameters/AffineMI_2d.txt')
sitk.PrintParameterMap( params_mi )

Notice the line `(Metric "AdvancedMattesMutualInformation")`, indicating that elastix will use mutual information. See the [documentation](https://elastix.lumc.nl/doxygen/classelastix_1_1AdvancedMattesMutualInformationMetric.html) if you are interested.

Now, run elastix with these parameters, and visualize the results by overlaying the images.

In [ ]:
elastix_mi = run_elastix(fixedImage, movingImage, params_mi)
result_mi = elastix_mi.GetResultImage()

In [ ]:
compare_overlay(result_mi, fixedImage)

While not perfect, the result is much better, with the skull of the CT and MR images nicely aligned.

## Bonus (optional): Mutual information details

Learn more about mutual information (and joint histograms) in the notebook `joint_histogram.ipynb`